#  Dataset Disponibilizado pela Olist

In [2]:
import pandas as pd

In [3]:
from sqlalchemy import create_engine
from decouple import config

HOST     = config('HOST')
DATABASE = config('DATABASE')
USER     = config('USER')
PASSWORD = config('PASSWORD')
PORT     = config('PORT')

string_conexao = f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}'
conn_olist = create_engine(string_conexao)

Forma de inserir/atualizar valores nas tabelas com o Pandas e com OS automaticamente

In [10]:
import os

lista = os.listdir('data')

for csv in lista:
    df_order_reviews = pd.read_csv('data/'+csv)
    df_order_reviews.to_sql(csv.replace('_database.csv','').replace('olist_',''), conn_olist, if_exists='replace', index=False)

In [19]:
#df_customers = pd.read_csv("Data/olist_customers_dataset.csv")
#df_customers.to_sql("customers", conn_olist, if_exists="replace")

441

Função para as consultas

In [4]:
def consulta(query):
    return pd.read_sql(query, conn_olist)

1) Faça uma consulta que retorne todos os vendedores do estado do Paraná.

In [5]:
consulta("SELECT * FROM sellers WHERE seller_state = 'PR'")

,index,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,9,ccc4bbb5f32a6ab2b7066a4130f114e3,80310,curitiba,PR
1,26,2078fe5066350e7d220c0ad3a3bbc6c1,86707,arapongas,PR
2,30,4dbd95a08b31ede5c82d3f861cb1ce1a,80210,curitiba,PR
3,37,d3e7d37c0df9aef383f3f2a15b0dddfb,83050,sao jose dos pinhais,PR
4,59,13c2ed7698b3ca92dad49e849219da59,84010,ponta grossa,PR
...,...,...,...,...,...
344,3065,5145090ab595c0d0b8557199f5701fbf,82900,curitiba,PR
345,3072,f90f77ef2799a27f80d90c425ca944f7,86020,londrina,PR
346,3073,bbf4a892cac7b82422baa711dffb8997,82200,curitiba,PR
347,3081,95cca791657aabeff15a07eb152d7841,85950,palotina,PR


2. Faça uma consulta que retorne todos os vendedores do estado do Paraná ou da cidade de sarandi.

In [6]:
consulta("SELECT * FROM sellers WHERE seller_state = 'PR' OR LOWER(seller_city) = 'sarandi'")

,index,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,9,ccc4bbb5f32a6ab2b7066a4130f114e3,80310,curitiba,PR
1,26,2078fe5066350e7d220c0ad3a3bbc6c1,86707,arapongas,PR
2,30,4dbd95a08b31ede5c82d3f861cb1ce1a,80210,curitiba,PR
3,37,d3e7d37c0df9aef383f3f2a15b0dddfb,83050,sao jose dos pinhais,PR
4,59,13c2ed7698b3ca92dad49e849219da59,84010,ponta grossa,PR
...,...,...,...,...,...
344,3065,5145090ab595c0d0b8557199f5701fbf,82900,curitiba,PR
345,3072,f90f77ef2799a27f80d90c425ca944f7,86020,londrina,PR
346,3073,bbf4a892cac7b82422baa711dffb8997,82200,curitiba,PR
347,3081,95cca791657aabeff15a07eb152d7841,85950,palotina,PR


3. Faça uma consulta que retorne todos os pedidos que o preço esteja entre R\$ 45 e R\$ 125

In [7]:
consulta('''SELECT order_id, payment_value
            FROM order_payments
            WHERE payment_value BETWEEN 45 AND 145
            ORDER BY payment_value
        ''')

,order_id,payment_value
0,b7fd5d833fed053c23020ca5b793e6de,45.0
1,4054f3a02c9191a518601fba04f06742,45.0
2,08597f2b3cec1c223f1b7950bda0a9fc,45.0
3,3a3f6aa80241caf165b717419017abc2,45.0
4,999b8d758771edb14fdf3f1a280d8a60,45.0
...,...,...
52638,e1a052031a62fc3bc7f2bf2d15241785,145.0
52639,c89a75d9ee5020cc2bdd4ebc2b35d2c3,145.0
52640,95c704fb2ac70f398a019de67a705ae8,145.0
52641,2cc3c6f1d89338c78360e079513f99d2,145.0


4. Faça uma consulta que retorne todos os pedidos calculando o valor total do pedido (preço + frete).

In [8]:
consulta('''SELECT order_id, (price + price + freight_value) AS total
            FROM order_items
            ORDER BY total
        ''')

,order_id,total
0,8272b63d03f5f79c56e9e4120aec44ef,10.29
1,8272b63d03f5f79c56e9e4120aec44ef,10.29
2,8272b63d03f5f79c56e9e4120aec44ef,10.29
3,8272b63d03f5f79c56e9e4120aec44ef,10.29
4,8272b63d03f5f79c56e9e4120aec44ef,10.29
...,...,...
112645,199af31afc78c699f0dbf71fb178d4d4,9454.34
112646,a96610ab360d42a2e5335a3998b4718a,9749.34
112647,f5136e38d1a14a4dbd87dff67da82701,13225.66
112648,fefacc66af859508bf1a7934eab1e97f,13651.21


5. Faça uma consulta que retorne a data mínima e máxima de limite de entrega

In [9]:
consulta('''SELECT MIN(shipping_limit_date), MAX(shipping_limit_date)
            FROM order_items
        ''')

,min,max
0,2016-09-19 00:15:34,2020-04-09 22:35:08


6. Faça uma consulta que retorne os 10 vendedores que mais realizaram vendas.

In [10]:
consulta('''SELECT seller_id, COUNT(DISTINCT order_id) as qtd
            FROM order_items
            GROUP BY seller_id
            ORDER BY qtd DESC
            LIMIT 10
        ''')

,seller_id,qtd
0,6560211a19b47992c3666cc44a7e94c0,1854
1,4a3ca9315b744ce9f8e9374361493884,1806
2,cc419e0650a3c5ba77189a1882b7556a,1706
3,1f50f920176fa81dab994f9023523100,1404
4,da8622b14eb17ae2831f4ac5b9dab84a,1314
5,955fee9216a65b617aa5c0531780ce60,1287
6,7a67c85e85bb2ce8582c35f2203ad736,1160
7,ea8482cd71df3c1969d7b9473ff13abc,1146
8,4869f7a5dfa277a7dca6462dcf3b52b2,1132
9,3d871de0142ce09b7081e2b9d1733cb1,1080


7. Faça uma consulta que retorne os tipos distintos de tipo de pagamento.

In [11]:
consulta('''SELECT DISTINCT payment_type
            FROM order_payments
        ''')

,payment_type
0,not_defined
1,boleto
2,debit_card
3,voucher
4,credit_card


8. Faça uma consulta que retorne a quantidade de pedidos agrupados por tipo de pagamento

In [12]:
consulta('''SELECT payment_type, COUNT(order_id) AS qtd
            FROM order_payments
            GROUP BY payment_type
            ORDER BY qtd
        ''')

,payment_type,qtd
0,not_defined,3
1,debit_card,1529
2,voucher,5775
3,boleto,19784
4,credit_card,76795


9. Faça uma consulta que retorne a soma dos valores do pedidos agrupados por tipo de pagamento

In [13]:
consulta('''SELECT payment_type, ROUND(CAST(SUM(payment_value) as numeric), 2) AS total
            FROM order_payments
            GROUP BY payment_type
            ORDER BY total
        ''')

,payment_type,total
0,not_defined,0.00
1,debit_card,217989.79
2,voucher,379436.87
3,boleto,2869361.27
4,credit_card,12542084.19


10. Faça uma consulta que retorne as 10 cidades com mais clientes.

In [23]:
consulta('''SELECT CONCAT(UPPER(customer_city), ' - ', customer_state) AS city_state,
            COUNT(DISTINCT customer_unique_id) AS qtd
            FROM customers
            GROUP BY city_state
            ORDER BY qtd DESC
            LIMIT 10
        ''')

,city_state,qtd
0,SAO PAULO - SP,14984
1,RIO DE JANEIRO - RJ,6620
2,BELO HORIZONTE - MG,2672
3,BRASILIA - DF,2069
4,CURITIBA - PR,1465
5,CAMPINAS - SP,1398
6,PORTO ALEGRE - RS,1326
7,SALVADOR - BA,1209
8,GUARULHOS - SP,1153
9,SAO BERNARDO DO CAMPO - SP,908


11. Faça uma consulta que crie uma nova coluna concatenando as colunas de cidade e estado separando por hífen.

In [24]:
consulta('''SELECT *, CONCAT(UPPER(customer_city), ' - ', customer_state) AS city_state
            FROM customers
        ''')

,index,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,city_state
0,0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,FRANCA - SP
1,1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,SAO BERNARDO DO CAMPO - SP
2,2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,SAO PAULO - SP
3,3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,MOGI DAS CRUZES - SP
4,4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,CAMPINAS - SP
...,...,...,...,...,...,...,...
99436,99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP,SAO PAULO - SP
99437,99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP,TABOAO DA SERRA - SP
99438,99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,FORTALEZA - CE
99439,99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,CANOAS - RS


12. Faça uma consulta que retorne a quantidade de ordens agrupadas por status do pedido

In [30]:
consulta('''SELECT order_status, COUNT(order_id) AS qtd
            FROM orders
            GROUP BY order_status
            ORDER BY qtd DESC
        ''')

,order_status,qtd
0,delivered,96478
1,shipped,1107
2,canceled,625
3,unavailable,609
4,invoiced,314
5,processing,301
6,created,5
7,approved,2


13. Faça uma consuta que calcule a diferença de tempo entre a compra e aprovação da realização do pedido.

In [39]:
consulta('''SELECT order_id id,
            (to_timestamp(order_approved_at, 'YYYY MM DD HH24 MI SS ') 
            - 
            to_timestamp(order_purchase_timestamp, 'YYYY MM DD HH24 MI SS ')) diferenca
            FROM orders
            ORDER BY diferenca DESC
        ''')

,id,diferenca
0,7a9d4c7f9b068337875b95465330f2fc,NaT
1,85e05c3c9aa9834518f35d4f286014be,NaT
2,2e9d8db9e132a9b5e1236618b7e8ef4a,NaT
3,b5359909123fa03c50bdb0cfed07f098,NaT
4,81b7c7bbc8ec003eeb67d87441a6a148,NaT
...,...,...
99436,a1dadce0e5cbb1b1205017166f6ce7ce,0 days
99437,26ba6dc5d33b55a3107a56f4e8d77395,0 days
99438,bdd02ebba4a073b670384d9ddb2a8b50,0 days
99439,4de6fae240d7acb7e1a830665d941ae2,0 days


In [45]:
consulta('''SELECT order_purchase_timestamp,  order_approved_at,  
            extract(day from date(order_approved_at))
            - extract(day from date(order_purchase_timestamp)) AS diff
            FROM orders
            ORDER BY diff DESC
        ''')

,order_purchase_timestamp,order_approved_at,diff
0,2018-03-05 22:57:35,None,NaN
1,2017-08-03 11:01:10,None,NaN
2,2018-09-03 14:14:25,None,NaN
3,2017-01-13 11:06:56,None,NaN
4,2017-08-18 14:30:19,None,NaN
...,...,...,...
99436,2018-05-31 13:13:17,2018-06-01 08:01:56,-30.0
99437,2018-07-31 10:29:51,2018-08-01 02:35:19,-30.0
99438,2017-08-31 11:49:09,2017-09-01 02:24:51,-30.0
99439,2018-07-31 21:51:41,2018-08-01 21:45:18,-30.0


In [42]:
consulta('''SELECT order_purchase_timestamp,  order_approved_at, 
            DATE(order_approved_at)
            -
            DATE(order_purchase_timestamp) AS diff
            FROM orders
            ORDER BY diff DESC
        ''')

,order_purchase_timestamp,order_approved_at,diff
0,2017-10-28 11:18:48,None,NaN
1,2017-09-27 20:03:11,None,NaN
2,2017-06-06 20:31:46,None,NaN
3,2017-02-18 14:40:00,None,NaN
4,2017-02-18 13:29:47,None,NaN
...,...,...,...
99436,2018-08-11 16:35:17,2018-08-11 16:50:25,0.0
99437,2018-02-19 12:41:45,2018-02-19 12:55:51,0.0
99438,2017-10-30 16:06:30,2017-10-30 17:31:35,0.0
99439,2018-04-27 16:51:52,2018-04-27 17:33:10,0.0


14. Faça uma consulta que retorne todos as reviews que tenham score maior ou igual a 3 e que não tenha conteúdo vazio no comentário da mensagem.

In [47]:
consulta('''SELECT *
            FROM order_reviews
            WHERE review_score >= 3 AND review_comment_message IS NOT NULL
            ORDER BY review_score DESC
        ''')

,index,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,10191,4b8c4a3edf44145ad7135370629ed25b,44e5e4871a2b86afa53f08e86001bca5,5,None,produto correto e entregue bem antes do prazo ...,2017-12-27 00:00:00,2017-12-28 00:06:39
1,4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
2,22300,a92c91484665dda9efe7167bca7a27ef,9e782eef55c0f6f0a4de48f967232e91,5,None,Comprei dois Moto G5 S plus com duas capas e d...,2017-10-14 00:00:00,2017-10-15 01:02:06
3,22301,37928401d9f3d87c84fabce8bc16b3cf,2efa8cad37f73bd607920b772a59b715,5,None,Produto maravilhoso.\r\nLojas lannister nota 10.,2018-01-13 00:00:00,2018-01-16 01:19:52
4,22303,dbd8e541b689bcc999f720a2432bb278,309c3acdc3ee0846604b0db436a91e94,5,None,Chegou no prazo! em ótimas condições,2017-08-16 00:00:00,2017-08-17 18:20:54
...,...,...,...,...,...,...,...,...
30082,93733,adc92c58a60ad8ab3a9bb76476dc320d,dbbe850a4424a2c03caa29c644644511,3,None,Atrasou a entrega,2017-08-05 00:00:00,2017-08-13 22:44:11
30083,97372,00c96cf39ea19ac6dec392d6b540e937,cefcc988df88b91e3c9825518535be06,3,None,"Ainda não posso dizer se o produto é bom, esto...",2017-10-03 00:00:00,2017-10-05 14:55:42
30084,96597,6d9236fabaaa4949f39847a70757a801,67f52d466554ffe40e2b3f2611c2a5a7,3,Não recomendo,Instalei primeiro o cartucho preto e ao que pa...,2018-06-08 00:00:00,2018-06-13 20:13:59
30085,87324,5bb6971cc3f8a16a55cc9048333daf48,bf6ae1909b2e970bb1c4f1119b324b64,3,None,FIQUEI TRISTE E ANSIOSO POIS O CÓDIGO DE RASTR...,2018-03-07 00:00:00,2018-03-14 02:34:08


In [48]:
consulta('''SELECT AVG(review_score)
            FROM order_reviews
        ''')
#Média das avaliações

,avg
0,4.086421


In [53]:
consulta('''SELECT *
            FROM order_reviews
            WHERE review_score >= (SELECT AVG(review_score) FROM order_reviews)
            AND review_comment_message IS NOT NULL
            ORDER BY review_score
        ''')
#Pedidos igual e acima da média

,index,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
1,4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
2,15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01
3,22,d21bbc789670eab777d27372ab9094cc,4fc44d78867142c627497b60a7e0228a,5,Ótimo,Loja nota 10,2018-07-10 00:00:00,2018-07-11 14:10:25
4,24,0e0190b9db53b689b285d3f3916f8441,79832b7cb59ac6f887088ffd686e1d5e,5,None,obrigado pela atençao amim dispensada,2017-12-01 00:00:00,2017-12-09 22:58:58
...,...,...,...,...,...,...,...,...
20549,99191,ed81d320e16c5999ff55348dce57d02b,daa7bd63b41e09a4d28579829649e6a2,5,None,Gostei muito.\r\nÉ a segunda vez que compro o ...,2017-08-20 00:00:00,2017-08-21 15:27:32
20550,99196,58be140ccdc12e8908ff7fd2ba5c7cb0,0ebf8e35b9807ee2d717922d5663ccdb,5,muito bom produto,"Ficamos muito satisfeitos com o produto, atend...",2018-06-30 00:00:00,2018-07-02 23:09:35
20551,99202,7047578adefcc7c27319e3cf91ba1416,01da5549f136167573c8a302f008557a,5,None,"Produto original,prazo de entrega rápido.Super...",2018-04-18 00:00:00,2018-04-22 14:48:57
20552,99208,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,5,None,Entregou dentro do prazo. O produto chegou em ...,2018-02-07 00:00:00,2018-02-19 19:47:23


15. Faça uma consulta que crie uma nova coluna com os meses de as datas de criação das reviews

In [58]:
consulta('''SELECT review_id, order_id,  
            extract(MONTH FROM DATE(review_creation_date)) AS mes
            FROM order_reviews
        ''')

,review_id,order_id,mes
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,1.0
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,3.0
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,2.0
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,4.0
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,3.0
...,...,...,...
99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,7.0
99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,12.0
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,3.0
99222,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,7.0


In [61]:
consulta('''SELECT COUNT(review_id) AS qtde_review,  
            extract(MONTH FROM DATE(review_creation_date)) AS mes
            FROM order_reviews
            GROUP BY mes
            ORDER BY qtde_review DESC
        ''')
#Contagem de reviews por mês

,qtde_review,mes
0,13490,8.0
1,11168,5.0
2,10284,3.0
3,10155,6.0
4,9342,4.0
5,9137,7.0
6,8027,12.0
7,7514,2.0
8,6418,1.0
9,4887,11.0
